In [3]:
####################################################################################################################
#Developed by Lukas Mueller (2023)
#CS-499 Capstone Project
#BS in Computer Science(conc. in Software Engineering)
####################################################################################################################

# Welcome to the MLDash project, which was written in the Spring of '23. As an aspiring machine learning engineer, 
#and software engineer by discipline, I recognize the value in making complicated tasks more accessible to the common
#user. Training machine learning algorithms requires tuning their (sometimes numerous) hyperparameters and hoping
#that the changes made boost model performance, while also keeping track of the changes that have been made.
#The main purpose of this project is to simplify this task for users, automate record keeping, and generate data
#visualizations that can assist users in the selection of more optimal hyperparameters. While the project currently
#hosts only one AI algorithm, OpenAI Gym's Cartpole-v1, I anticipate that additional models can be added as modules
#over time. The cartpole algorithm is deep-Q reinforcement learning algorithm that solves the problem of balancing
#a pole attached to cart that moves on a frictionless rail. The model acquires data from the environment
#(observation space) and determines which sequence of actions results in better performance for each situation. 
#The model trains itself by replaying experiences it acquires, prioritizing samples of its experience based on how 
#different those experiences were from its predictions. The model begins training with a higher likelyhood to explore,
#allowing it to gather new experiences by trying numerous different action sequences; as training continues, the model
#relies less on exploration and more on the exploitation of the experiences it has gathered. 

#If you are reading this, you are looking at the central file for the project. This file creates an instance of the 
#MLMongo class (defined in crud.py) to initiate a connection to a local MongoDB service. 
#It also creates an instance of the Cartpole class, which makes use of the DQNsolver(for the learning model setup, and 
#learning procedures), PrioritizedBuffer(an upgraded sampling procedure that makes experience replay a non arbitrary 
#process), and ScoreLogger(used to record hyperparameter profiles along with key performance metrics) classes. During 
#training, hyperparameter values and performance metrics are written to the metrics.csv file. The metrics 
#collection and summary collection are contained in the MongoDB TRAIN database. When the model successfully balances 
#the pole (for a minimum average duration) the metrics.csv file that contains training data is read into a Python 
#dictionary and written to the TRAIN/metrics database collection. Once the database has been populated with training 
#data, users can view this data using the dashboard. The dashboard allows the data to be viewed in the form of an 
#interactive data table and dynamic data visualizations. Users have the ability to filter data to examine summary 
#data and individual training sessions, and the accompanying charts change along with the datatable.

#This projec was created by integrating a simpler dashboard program and a basic cartpole learning model implementation.
#Various upgrades were made, along the way, in areas related to software engineering, algorithms and data structures,
#and data structures:

#The names of files, functions, and variables were updated to reflect the new context

#Dependencies were updated to reflect more recent versioning

#Significant amounts of documentation was added in the form of comments and a readme file.

#Code was integrated and refactored into a more modular form that relies on seperate classes.

#Additional functions were added to the crud.py, score_logger.py to facilitate local storage of learning model training
#data, and the ability to write this data to a database. The Dashboard.ipynb file was upgraded to provide a relevant 
#and task-specific interface with enhanced interactivity, dataframe filtering, data session indexing, and data 
#visualization capabilities.

#The learning algorithm itself enjoyed a significant upgrade that improved its ability to learn from its experiences.
#During exprience replay (code found in dqn_solver.py) the batch sampling method was upgraded from using a random
#sampling procedure to use prioritized sampling (code found in the prioritized_buffer.py file), resulting in a more
#effective implementation of the learning algorithm and contributing to significant reduction in total memory use. 

#By using tool that helps cultivate data perspective and insights, the process of training a deep-Q reinforcement
#learning algorithm becomes more intuitive for users, while reducing the cognitive burden associated with analyzing
#huge amounts of data. Enjoy!
###################################################################################################################


#Dashboard imports

from jupyter_dash import JupyterDash
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
#from dash import Button
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
from bson.json_util import dumps
from bson.objectid import ObjectId
import base64
from crud import MLMongo
from cartpole import MLCartpole
import statsmodels
import csv

#########################
# CONSTANTS (file references)
#########################
METRICS_FILE = "metrics.csv"
SUMMARY_FILE = "summary.csv"
IMAGE_FILE = 'logo.png'   

#########################
# Session count
#########################

session_count = 0
      
#Helper functions                               
def count_sessions():
    global session_count
    dFrame = pd.DataFrame.from_records(mongobject.read_all({}))
    dFrame = dFrame.iloc[:,0:]
    try:
        df_sorted = dFrame.sort_values(by='session', ascending=False)
        top_session = df_sorted.head(1)['session'] 
        session_count = top_session.item()
        if session_count == None:
            session = 0
    except:
        session_count = 0

    return int(session_count)
     

def increment_session():
    global session_count
    session_count = count_sessions() + 1 
        
    return session_count


###########################
# Data Manipulation / Model
###########################

username = str('aiuser')
password = str('password')

#Creates the MongoDB CRUD-capable instance
mongobject = MLMongo(username, password)

# class read method must support return of cursor object (cursor refers to the entire metrics collection)
df = pd.DataFrame.from_records(mongobject.read_all({}))
df = df.iloc[:,0:]

                               
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('__name__')

#reads a local image file and encodes it as a base64 string. 
#The open() function is used to open the file in binary mode, 
#and base64.b64encode() is used to encode the binary data as a base64 string.
#(displays images embedded in the app layout using the html.Img() component)
encoded_image = base64.b64encode(open(IMAGE_FILE, 'rb').read())

    
# The layout displays a logo and title, followed by a grid layout that accepts user input in multiple ways
# Once the database is populated, the input section is followed by an datatable
# Beneath the datatable, data is visualized using several charts, which are dynamically updated to reflect visible datatable data
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Machine Learning Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H2("Developed by Lukas Mueller (2023)"))),
    html.Hr(),
    
    html.Div([
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                 html.Div(),
                 html.Div(
                     html.Label('Alpha:'),
                     className='col s12 m6'
                     ),
                 html.Div(
                     dcc.Input(id='input-1', type='number', value= 0.01, placeholder="0>x>1, low"),
                     className='col s12 m6'
                     ),
                 html.Div(style={'text-align': 'right', 'margin-right': '10px'},
                          children = [
                              html.Div(id='reduce_to_rubble'),
                              html.Button('Wipe DB', id = 'wipe-database-button', n_clicks = 0)
                          ])
             ]),
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                 html.Div(),
                 html.Div(
                     html.Label('Gamma:'),
                     className='col s12 m6'
                     ),
                 html.Div(
                     dcc.Input(id='input-2', type='number', value= 0.90, placeholder="0>x>1, high"),
                     className='col s12 m6'
                     ),
                 html.Div()
                 
             ]),
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                 html.Div(),
                 html.Div(
                     html.Label('Epsilon Min:'),
                     className='col s12 m6'
                 ),
                 html.Div(
                     dcc.Input(id='input-3', type='number', value= 0.01, placeholder="0>x>1, low"),
                     className='col s12 m6'
                 ),
                 html.Div(),
                 
             ]),
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                html.Div(),
                html.Div(
                    html.Label('Epsilon Max:   '),
                    className='col s12 m6'
                    ),
                html.Div(
                    dcc.Input(id='input-4', type='number',  value= 0.9, placeholder="0>x>1, high"),#alt value=0.75
                    className='col s12 m6'
                    ),
                html.Div(),
                 
             ]),
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                html.Div(),
                html.Div(
                    html.Label('Epsilon Decay: '),
                    className='col s12 m6'
                    ),
                html.Div(
                    dcc.Input(id='input-5', type='number', value= 0.90, placeholder="0>x>1, high"),#alt value=0.95 
                    className='col s12 m6'
                    ),
                html.Div()
                 
             ]),
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                html.Div(),
                html.Div(
                    html.Label('# of Episodes: '),
                    className='col s12 m6'
                    ),
                html.Div(
                    dcc.Input(id='input-6', type='number', value= 20, placeholder="10-40"),
                    className='col s12 m6'
                    ),
                html.Div()
                
             ]),
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                html.Div(),
                html.Div(
                    html.Label('Batch Size: '),
                    className='col s12 m6'
                    ),
                html.Div(
                    dcc.Input(id='input-7', type='number', value=40, placeholder="Default value: 20"),#alt value=20
                    className='col s12 m6'
                    ),
                
                html.Div()
             ]),
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                 html.Div(),
                 html.Div(
                     html.Label('Buffer Size: '),
                     className='col s12 m6'
                     ),
                 html.Div(
                     dcc.Input(id='input-8', type='number', value=10000, placeholder="Default value: 10000"),
                     className='col s12 m6'
                     ),
                 
                 html.Div(
                     html.Label("Latest Training Session #: ")
                 )
             ]),
            
        
        html.Div(className='row',
         style={'display' : 'grid', 'grid-template-columns': '1fr 1fr 1fr 1fr', 'grid-gap': '25px'},
             children=[
                 html.Div(),
                 html.Div(),
                 html.Div(style={'text-align': 'center', 'margin-top': '10px', 'margin-right': '10px'},
                    # The submit button saves the hyperparameter values the user enters and uses them to create an instance of the 
                    # Cartpole class. This initiates a reinforcement learning training session.
                     children = [
                         html.Div(id='output'),
                         html.Button('Submit', id='submit-button', n_clicks=0)
                     ]),
                 html.Div(
                     html.Label(id="session-total",
                        children = str(session_count)
                     )
                 )
             ])
       
                
    ]),
    
    html.Hr(),
    html.Br(),
    
    #this row houses four radio buttons that are used to filter the data
    html.Div(className='row',
        style={'display': 'flex'},
            children=[
                dcc.RadioItems(
                    id="filter-type",
                    #Labels are provisioned for useful filtering of performance metrics
                    options=[
                       {'label': 'High Scores', 'value': 'highscore'},
                       {'label': 'Low Runs', 'value': 'lowrun'},
                       {'label': 'Both', 'value': 'both'},
                       {'label': 'All', 'value': 'all'}
                    ],
                    value='reset'
                )
            ]
    ),

    html.Hr(),
    #this is the layout for the data table
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        
        #these are options which mostly provide for native interactivity with the table
        data=df.to_dict('records'),
        editable= False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable= False,
        row_selectable= False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=1000,

        
        #Below is some simple styling for the data table to make it easier to look at...
        
        #table striping implemented to make visual tracking of a document's data easier
        style_cell_conditional=[
            {
                'if': {'column_id': c},
                'textAlign': 'left'
            } for c in ['Date', 'Region']
        ],
        style_data={
            'color': 'black',
            'backgroundColor': 'white'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(220, 220, 220)',
            }
        ],
        style_header={
            'backgroundColor': 'rgb(210, 210, 210)',
            'color': 'black',
            'fontWeight': 'bold'
        },
        #sizing for table cells, to increase presentability based on length of data
        style_cell={
            'minHeight': '16px', 'height': '16px', 'maxHeight': '16px',
            'minWidth': '160px', 'width': '160px', 'maxWidth': '160px',
            'whiteSpace': 'normal'
        },
    ),
    
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that two charts are diplayed per row
    # Row 1 (all sessions)
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                #this is the first chart: pie chart (correlation between session and total runs)
                html.Div(
                    id='graph1-id',
                    className='col s12 m6',
                    ),
                #this is the second chart: bar chart  (correlation between session and total runs)
                html.Div(
                    id='graph2-id',
                    className='col s12 m6',
                    )
             ]),
    # Row 2 (filter-specific)
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                #this is the third chart: scatterplot with linear regression (correlation between runs and max score)
                html.Div(
                    id='graph3-id',
                    className='col s12 m6',
                    ),
                #this is the second chart: scatterplot with lowess trendline (correlation between runs and max score)
                html.Div(
                    id='graph4-id',
                    className='col s12 m6',
                    )
             ]),
    #Row 3 (filter-specific)
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                #this is the first chart: scatterplot with linear regression (correlation between runs and exploration rate)
                html.Div(
                    id='graph5-id',
                    className='col s12 m6',
                    ),
                #this is the second chart: scatterplot with lowess trendline (correlation between runs exploration rate)
                html.Div(
                    id='graph6-id',
                    className='col s12 m6',
                    )
             ]),
    #Row 4 (filter-specific)
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                #this is the first chart: scatterplot with linear regression (correlation between exploration rate and max score)
                html.Div(
                    id='graph7-id',
                    className='col s12 m6',
                    ),
                #this is the second chart: scatterplot with lowess trendline (correlation between exploration rate and max score)
                html.Div(
                    id='graph8-id',
                    className='col s12 m6',
                    )
             ])
])




#############################################
# Interaction Between Components / Controller
#############################################

#This function can be used to store user input values for hyperparameter to local variables, but in this case they are used ("on-the-fly") 
# as function arguments. This causes the cartpole function execute as soon as the user submits their input data. 
@app.callback(Output('output', 'children'),
              [Input('submit-button', 'n_clicks')],
              [State('input-1', 'value'),
               State('input-2', 'value'),
               State('input-3', 'value'),
               State('input-4', 'value'),
               State('input-5', 'value'),
               State('input-6', 'value'),
               State('input-7', 'value'),
               State('input-8', 'value')])         
   
def update_output(n_clicks, input1, input2, input3, input4, input5, input6, input7, input8):
    global session_count
    solved = False
    if n_clicks > 0:
        print("Please wait while the learning algorithm trains. Performance metrics will then be viewable.") 
        
        #increment session count to tag this session with a session id equal to stored sessions + 1
        session_count = increment_session()
        
        cartpole_instance = MLCartpole(session_count, input1, input2, input3, input4, input5, input6, input7, input8)
        
        solved = cartpole_instance.cartpole()
        
        n_clicks = 0
        
        if solved == True:
            print("This model was successful. Congratulations! Writing to database...")
             
            try:
                mongobject.write_to_metrics()
            except:
                print("There was an issue preventing records from being written to the database. Please contact your IT Administrator, local developer, and nearest coffee house.")
            
            try:
                mongobject.write_to_summary()
                
            except:
                print("There was an issue writing summary data to the database. So Sad...")
                
            update_dashboard('reset')
            return f'Dashboard updated: {solved}'
        
        else:
            print("This version exceeded the run limit. Enter a different set of values, and try again")
            return ''  



# This function deletes all data stored in the metrics and summary collections in the TRAIN database.
# It used another function defined in crud.py, but has been repackaged here to occur when the user clicks on the "Wipe DB" button
@app.callback(Output('reduce_to_rubble', 'children'),
              [Input('wipe-database-button', 'n_clicks')])

def wipe_stored_data(n_clicks):
    if n_clicks > 0:
        mongobject.clear_collections()    
    
    
#Radio filter callback, with different options to filter for learning algorithm params. 
#Reset filter removes other filters to display all data.
@app.callback([Output('datatable-id','data'), Output('datatable-id','columns'), Output('session-total', 'children')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    global session_count
    df = pd.DataFrame.from_records(mongobject.read_all({}))
    df = df.iloc[:,0:]
    
    # Update the displayed session count
    session_count = count_sessions()
        
    # Construct the new label text
    updated_session_count = html.Label(id="session-total", children=str(session_count))
    
    

    #provides the search query for learning rate parameters and the corresponding time to solve
    if filter_type == 'highscore':
        df = pd.DataFrame.from_records(mongobject.read_all({
            "session" :  {"$gte":0, "$lte":session_count},
            "alpha" : {"$gte":0.001, "$lte":0.01},
            "run" : {"$gte":100, "$lte":3000},
            "meanScore": {"$gte": 0},
            "maxScore": {"$gte":100, "$lte":1000}
        }))
        df = df.iloc[:,0:]

    #provides the search query for parameter values related to planning incentivization and the corresponding time to solve
    elif filter_type == 'lowrun':
        df = pd.DataFrame.from_records(mongobject.read_all({
            "session" :  {"$gte":0, "$lte":session_count},
            "alpha" : {"$gte":0.001, "$lte":0.01},
            "run" : {"$gte":100, "$lte":2000},
            "meanScore": {"$gte":0 },
            "maxScore": {"$gte":0, "$lte":1000}
        }))
        df = df.iloc[:,0:]

    #provides the search query for exploration parameter values and the corresponding time to solve
    elif filter_type == 'both':
        df = pd.DataFrame.from_records(mongobject.read_all({
            "session" : {"$gte":0, "$lte":session_count},
            "alpha" : {"$gte":0.001, "$lte":0.01},
            "run" : {"$gte":100, "$lte":2000},
            "meanScore": {"$gte": 0},
            "maxScore": {"$gte":100, "$lte":1000}
        }))
        df = df.iloc[:,0:]

    # Reset query filter by querying all the data without specifying any particulars
    elif filter_type == 'all':
        df = pd.DataFrame.from_records(mongobject.read_all({}))
        df = df.iloc[:,0:]
        
    
    # store the columns and the queried data in the columns and data variable, and return to enable display
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    #df = df.iloc[:,0:]
    data=df.to_dict('records')
    
    return data,columns,[updated_session_count]


#Highlight any column that is selected via checkboxes displayed in header cells
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])

def update_style_c(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



####################################
# Update all charts
####################################

#These first two charts display summary data from all data in the dataframe, making them less responsive to filtering
# Updates the first chart
@app.callback(Output('graph1-id', "children"),
              [Input('datatable-id', "derived_viewport_data")])


def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #1."
    
    viewData = {'session': [row['session'] for row in viewData]}
    
    dff = pd.DataFrame.from_dict(viewData)    
    fig = px.pie(dff, names ='session') 
    pieChart = [dcc.Graph(figure=fig)]
    
    return pieChart

# Updates the second chart
@app.callback(Output('graph2-id', "children"),
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #2."
    
    viewData = {'session': df['session'].tolist(),'run': df['run'].tolist()}
    
    dff = pd.DataFrame.from_dict(viewData) 
    fig = px.bar(dff, x='session', y='run', labels={'x': 'session', 'y': 'run'})
    barChart = [dcc.Graph(figure=fig)]
    return barChart


#The rest of the charts change to show data that is in the current data table, only
#This makes them responsive to filtering and search functionality

# Updates the third chart (run vs max score)
@app.callback(Output('graph3-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #3."

    
    # Convert viewData dictionary to a list of rows, skipping the first rows (header)
    rows = [[row['run'], row['maxScore']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'maxScore'])
    
    
    fig = px.scatter(dff, x='run', y='maxScore', labels={'x': 'Runs', 'y': 'Max Score'})
    
    
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot

# Updates the fourth chart (run vs max score, with trendline)
@app.callback(Output('graph4-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #4."

    
    # Convert viewData dictionary to a list of rows, skipping the first rows (header)
    rows = [[row['run'], row['maxScore']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'maxScore'])
    
    # Filter out rows with zero 'run' values
    dff_filtered = dff[dff['run'] != 0]
    
    #drop rows with NaN values
    dff_filtered.dropna()
    
    fig = px.scatter(dff_filtered, x='run', y='maxScore', labels={'x': 'Runs', 'y': 'Max Score'}, trendline="ols")
    
    
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# Updates the fifth chart (run vs explore rate)
@app.callback(Output('graph5-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #5."

    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['run'], row['exploreRate']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'exploreRate'])

    
    fig = px.scatter(dff, x='run', y='exploreRate', labels={'x_column': 'Runs', 'y_column': 'Experience Based/Exploration'})
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot

# Updates the sixth chart (run vs explore rate, with trendline)
@app.callback(Output('graph6-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #6."

    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['run'], row['exploreRate']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['run', 'exploreRate'])
  
    # Filter out rows with zero 'run' values
    dff_filtered = dff[dff['run'] != 0]
    
    #drop rows with NaN values
    dff_filtered.dropna()
    
    fig = px.scatter(dff_filtered, x='run', y='exploreRate', labels={'x_column': 'Runs', 'y_column': 'Experience Based/Exploration'}, trendline ="ols")
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot


# Updates the seventh chart (exploration vs maximum score)
@app.callback(Output('graph7-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #7."

    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['exploreRate'], row['maxScore']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['exploreRate', 'maxScore'])
    

    fig = px.scatter(dff, x='exploreRate', y='maxScore', labels={'x_column': 'Experience Based/Exploration', 'y_column': 'Maximum Score'})
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot

# Updates the eighth chart (exploration vs maximum score with trendline)
@app.callback(Output('graph8-id', "children"), 
              [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    if not viewData:
        return "No data available in viewport #8."
    
    
     # Convert viewData dictionary to a list of rows, skipping the first row (header)
    rows = [[row['exploreRate'], row['maxScore']] for i, row in enumerate(viewData) if i>1]
    
    # Create a DataFrame from the data and specify the column names
    dff = pd.DataFrame(rows, columns=['exploreRate', 'maxScore'])

    # Filter out rows with zero 'run' values
    dff_filtered = dff[dff['exploreRate'] != 0.0]
    
    #drop rows with NaN values
    dff_filtered.dropna()
    
    fig = px.scatter(dff_filtered, x='exploreRate', y='maxScore', labels={'x_column': 'Experience Based/Exploration', 'y_column': 'Maximum Score'}, trendline = "ols")
    scatterPlot = [dcc.Graph(figure=fig)]
    return scatterPlot



# This displays the app content in Jupyter Notebook; A link appears in the notebook that allows the user to opt for 
# a full-screen view after execution
# of the application in their default web browser (http://localhost:8890)
if __name__ == '__main__':
    app.run_server(mode='inline', port=8890)

Dash is running on http://127.0.0.1:8890/

